In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt

# Initialize Spark Session
spark = SparkSession.builder.appName("HackerNewsClustering").getOrCreate()

# Load and sample data
data = spark.read.csv('story.csv', inferSchema=True, header=True)
sampled_data = data.sample(False, 0.0001)  # Adjust the fraction to manage the dataset size

# Preprocess the dataset
# Replace with actual embedding column names if they are different
assembler = VectorAssembler(inputCols=[f'embed_{i}' for i in range(1, 1537)], outputCol="features")
dataset = assembler.transform(sampled_data)

# Elbow method to determine optimal K
cost = []
evaluator = ClusteringEvaluator()
for k in range(2, 10):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(dataset)
    predictions = model.transform(dataset)
    cost.append(evaluator.evaluate(predictions))

# Plotting the Elbow Method result
plt.plot(range(2, 10), cost)
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Elbow Method')
plt.show()

# Silhouette method
optimal_k = cost.index(max(cost)) + 2
print(f"Optimal number of clusters: {optimal_k}")

# Clustering using optimal K
kmeans = KMeans().setK(optimal_k).setSeed(1).setFeaturesCol("features")
model = kmeans.fit(dataset)
predictions = model.transform(dataset)

# Evaluate clustering
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette with optimal K: {silhouette}")


23/12/11 22:51:54 WARN Utils: Your hostname, Hadis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.10.5 instead (on interface en0)
23/12/11 22:51:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/11 22:51:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/11 22:51:59 ERROR Executor: Exception in task 1.0 in stage 1.0 (TID 2) 18]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at org.apache.hadoop.io.Text.setCapacity(Text.java:266)
	at org.apache.hadoop.io.Text.append(Text.java:236)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:254)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.Uncompr

ConnectionRefusedError: [Errno 61] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
